In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
from typing import TypedDict


class DataType(TypedDict):
    X_train: np.ndarray
    X_test: np.ndarray
    Y_train: np.ndarray
    Y_test: np.ndarray
data: DataType = np.load("data.npy")

In [4]:
model = keras.models.load_model("./model.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense1 (Dense)              (None, 10)                100       
                                                                 
 dense2 (Dense)              (None, 16)                176       
                                                                 
 dense3 (Dense)              (None, 32)                544       
                                                                 
 dense4 (Dense)              (None, 4)                 132       
                                                                 
 output (Dense)              (None, 1)                 5         
                                                                 
Total params: 957
Trainable params: 957
Non-trainable params: 0
_________________________________________________________________


In [6]:
def representative_data_gen():
  for input_value in data['X_train'].astype("float32"):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\JORGE~1.RUI\AppData\Local\Temp\tmp4arbt_vk\assets


INFO:tensorflow:Assets written to: C:\Users\JORGE~1.RUI\AppData\Local\Temp\tmp4arbt_vk\assets
c:\Users\jorge.ruiz\Anaconda3\envs\quant\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [7]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.float32'>
output:  <class 'numpy.float32'>


In [8]:
import pathlib
tflite_model_quant_file = "model.tflite"
with open(tflite_model_quant_file, "wb") as f:
    f.write(tflite_model_quant)

In [24]:
predictions = []

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
interpreter.allocate_tensors()



for x in data['X_test'].astype("float32"):

    input_data = np.expand_dims(x, axis=1).astype(input_details["dtype"])
    input_data = np.array([x])

    interpreter.set_tensor(input_details["index"], input_data)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    predictions.append(output[0])



In [26]:
np.square(np.array(predictions) - data['Y_test']).mean()

0.022257305721855743

In [ ]:
0.022209661081433296